In [1]:
%cd /content/drive/MyDrive/데청캠 5조/Test_1

/content/drive/MyDrive/데청캠 5조/Test_1


In [2]:
!pwd

/content/drive/MyDrive/데청캠 5조/Test_1


In [3]:
import os
import cv2
import numpy as np
from moviepy.editor import VideoFileClip
import datetime
import time
from tqdm import notebook
import re
import glob

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3031040/45929032 bytes (6.6%)6930432/45929032 bytes (15.1%)10911744/45929032 bytes (23.8%)14958592/45929032 bytes (32.6%)18735104/45929032 bytes (40.8%)22380544/45929032 bytes (48.7%)25919488/45929032 bytes (56.4%)29638656/45929032 bytes (64.5%)33546240/45929032 bytes (73.0%)37527552/45929032 bytes (81.7%)41484288/45929032 bytes (90.3%)45465600/45929032 bytes (99.0%)

In [4]:
# MOV_List : 영상 데이터 저장하는 변수
MOV_List = np.array(glob.glob('./MOV/*.mp4'))
MOV_List

array(['./MOV/readnaversid1750oid448aid0000195576.mp4', './MOV/video.mp4',
       './MOV/readnaversid1750oid448aid0000196603.mp4',
       './MOV/readnaversid1750oid448aid0000310334.mp4',
       './MOV/readnaversid1750oid448aid0000195268.mp4',
       './MOV/readnaversid1742oid437aid0000011423.mp4'], dtype='<U45')

In [5]:
# Two_min_under_MOV 함수 : 2분 이하 동영상 선별 

def Two_min_under_MOV(MOV_List):
    
    # Using_List : 사용할 영상들 file_name 모음 
    Using_List = MOV_List[:]
    
    # del_index : 삭제할 영상들 index 모음
    del_index = np.array([])
    del_index = del_index.astype(int)
    
    for i in notebook.tqdm(range(len(MOV_List))):
        clip = VideoFileClip(MOV_List[i])
        play_time = clip.duration
        
        # 재생 시간이 2분을 넘는 경우 -> Using_list에서 제외
        if play_time > 60*2 :
            print('Except MOV_file name :', Using_List[i])
            del_index = np.append(del_index, [i])
            
    Using_List = np.delete(Using_List, del_index)
            
    # 2분 미만인 동영상 리스트
    return Using_List

In [6]:
Using_MOV_List = Two_min_under_MOV(MOV_List)
print(Using_MOV_List)

Except MOV_file name : ./MOV/readnaversid1750oid448aid0000195268.mp4

['./MOV/readnaversid1750oid448aid0000195576.mp4' './MOV/video.mp4'
 './MOV/readnaversid1750oid448aid0000196603.mp4'
 './MOV/readnaversid1750oid448aid0000310334.mp4'
 './MOV/readnaversid1742oid437aid0000011423.mp4']


In [7]:
# one_anchor_starts_MOV 함수 : 앵커 1명의 얼굴로 시작되는 영상 리스트 찾기
def one_anchor_starts_MOV(MOV_List):
     
    Using_MOV = MOV_List[:]
    
    # 가중치 파일 경로
    cascade_filename = 'haarcascade_frontalface_alt.xml'
    # 모델 불러오기
    cascade = cv2.CascadeClassifier(cascade_filename)
    
    # 첫 장면에서 인식한 얼굴의 수
    face_num_list = np.array([])
    face_num_list = face_num_list.astype(int)
    
    for i in notebook.tqdm(range(len(MOV_List))):
        video = cv2.VideoCapture(MOV_List[i]) #video name change
        ret, frame = video.read()
        
        # 첫 번째로 등장하는 프레임만 취급
        frame[0]

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        results = cascade.detectMultiScale(gray,
                                          scaleFactor=1.2,
                                          minNeighbors=5, minSize=(20,20))
        
        face_num_list = np.append(face_num_list, [len(results)])
    
    # del_index : 삭제할 영상들 index 모음
    del_index = np.array([])
    del_index = del_index.astype(int)
    
    # 앵커 1명으로 시작하는 영상 찾기
    for k in range(len(face_num_list)):
        if face_num_list[k]!=1:
            del_index = np.append(del_index, [k])
    
    Using_MOV = np.delete(Using_MOV, del_index)
    
    # 파일 처리 쉽게 할 수 있는 것을 목표 
    # -> MOV 원본 파일 주소, MOV 파일확장자 지운 버전 모두 제공 
    return Using_MOV

In [8]:
Using_MOV_List = one_anchor_starts_MOV(Using_MOV_List)
print(Using_MOV_List)


['./MOV/readnaversid1750oid448aid0000195576.mp4'
 './MOV/readnaversid1750oid448aid0000310334.mp4'
 './MOV/readnaversid1742oid437aid0000011423.mp4']


In [9]:
# Clean_File_List 변수 : 소속 위치, 확장자 뗀 본연의 파일명 저장
Clean_File_List = np.array([])

for File in Using_MOV_List:
  File_name = re.sub(pattern='./MOV/', repl='', string=File)
  File_name = re.sub(pattern='.mp4', repl='', string=File_name)
  Clean_File_List = np.append(Clean_File_List, [File_name])

print(Clean_File_List)

['readnaversid1750oid448aid0000195576'
 'readnaversid1750oid448aid0000310334'
 'readnaversid1742oid437aid0000011423']


In [10]:
!pip install git+https://github.com/pyannote/pyannote-audio

  Cloning https://github.com/pyannote/pyannote-audio to /tmp/pip-req-build-s_blx83_
  Running command git clone -q https://github.com/pyannote/pyannote-audio /tmp/pip-req-build-s_blx83_
     |████████████████████████████████| 56 kB 3.4 MB/s 
     |████████████████████████████████| 41 kB 331 kB/s 
     |████████████████████████████████| 49 kB 7.5 MB/s 
     |████████████████████████████████| 129 kB 28.8 MB/s 
     |████████████████████████████████| 302 kB 57.1 MB/s 
     |████████████████████████████████| 80 kB 11.1 MB/s 
     |████████████████████████████████| 164 kB 66.8 MB/s 
     |████████████████████████████████| 75 kB 5.7 MB/s 
     |████████████████████████████████| 49 kB 7.5 MB/s 
     |████████████████████████████████| 141 kB 73.5 MB/s 
     |████████████████████████████████| 111 kB 72.8 MB/s 
  Created wheel for pyannote.audio: filename=pyannote.audio-1.1.2+3.g106dce8-py3-none-any.whl size=231383 sha256=720411f0df42003c767baaf6cda4e81ee6a48d674179674ca71e0e43007ddd06
  Stored 

In [11]:
# MOV_to_WAV 함수 : 영상(.mp4) -> 음성(.wav) 전환 작업
def MOV_to_WAV(MOV_List):
    WAV_file = np.array([])
    
    for MOV in notebook.tqdm(MOV_List): 
        clip = VideoFileClip(MOV)

        # 파일 저장 위치 변경
        sound_file_name = re.sub(pattern='MOV', repl='WAV', string=str(MOV))
        # 파일 저장 형태 변경
        sound_file_name = re.sub(pattern='mp4', repl='wav', string=sound_file_name)
        
        clip.audio.write_audiofile(sound_file_name)

In [12]:
MOV_to_WAV(Using_MOV_List)

[MoviePy] Writing audio in ./WAV/readnaversid1750oid448aid0000195576.wav


100%|██████████| 1677/1677 [00:01<00:00, 1161.71it/s]

[MoviePy] Done.


[MoviePy] Writing audio in ./WAV/readnaversid1750oid448aid0000310334.wav


100%|██████████| 2057/2057 [00:01<00:00, 1178.90it/s]

[MoviePy] Done.


[MoviePy] Writing audio in ./WAV/readnaversid1742oid437aid0000011423.wav


100%|██████████| 2044/2044 [00:02<00:00, 967.00it/s]


[MoviePy] Done.



In [13]:
import torch
import pyannote
from pyannote.database import get_protocol
from pyannote.database import FileFinder
from pyannote.audio.utils.signal import Binarize

from datetime import datetime, timedelta
import time

In [14]:
# sad 모델 설정
sad = torch.hub.load('pyannote/pyannote-audio', 'sad_ami')
preprocessors = {'audio':FileFinder()}
protocol = get_protocol('AMI.SpeakerDiarization.MixHeadset',
                            preprocessors=preprocessors)
    
binarize = Binarize(offset=0.52, onset=0.52, log_scale=True,
                    min_duration_off=0.03, min_duration_on=0.03)

Downloading: "https://github.com/pyannote/pyannote-audio/archive/master.zip" to /root/.cache/torch/hub/master.zip


In [15]:
# Usig_sad 함수 : sad 모델 사용 함수
def Using_sad(WAV_File):
  test_file = {}
  test_file['audio'] = WAV_File
  sad_scores = sad(test_file)

  speech = binarize.apply(sad_scores, dimension=1)

  # TimeStamp 편집 과정
  a = str(speech[0])
  a = re.sub(pattern='\-\-\>', repl='', string=a)
  a = re.sub(pattern='\[', repl='', string=a)
  a = re.sub(pattern='\]', repl='', string=a)
  a = str.split(a)
  
  b = a[1]
  b = b.split(sep=':')
  
  target_sec = 0
  index = 0
  for i in reversed(b):
    target_sec += float(i) * (60**index)
    index += 1
  print(WAV_File, ':', target_sec, 'sec')

  return WAV_File, target_sec        

In [16]:
Using_sad('./WAV/readnaversid1742oid437aid0000011423.wav')

./WAV/readnaversid1742oid437aid0000011423.wav : 10.038 sec


('./WAV/readnaversid1742oid437aid0000011423.wav', 10.038)

In [17]:
!pip install pydub

In [18]:
!pip install speechrecognition

     |████████████████████████████████| 32.8 MB 35 kB/s 


In [19]:
import moviepy.editor as mp
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import re
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import os
import speech_recognition as sr

In [20]:
def face_crop_vid (Clean_File_name):
    
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    File_name = './CUT_VID/' + Clean_File_name + '.mp4'
    video =  cv2.VideoCapture(File_name)

    fps = video.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    width = video.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = video.get(cv2.CAP_PROP_FRAME_HEIGHT)
    size = (round(width), round(height))
    out = cv2.VideoWriter('./FINAL_VID/%s.mp4' %Clean_File_name, fourcc, fps, size)

    while(video.isOpened()):
    
        ret, frame = video.read()
        
        if frame is None:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 
                                              scaleFactor=1.2, 
                                              minNeighbors=5, 
                                              minSize=(20,20))


        for (x, y, w, h) in faces:
            cropped = frame[y - int(h/4):y + h + int(h/4), x - int(w/4):x + w + int(w/4)]


        out.write(cropped)

    out.release()


In [21]:
## 파일 전체 대상으로 코드 돌리기 
# (문장 단위로 끊기에는 문장과 문장을 연달아 하는 발음이 많아서 앵커 대사 기준으로 끊어봤음)

for File in notebook.tqdm(Clean_File_List):
  # 시간 좌표 구하기 (통문장)
  WAV_File = './WAV/' + File + '.wav'
  print(WAV_File)
  WAV_File, target_sec = Using_sad(WAV_File)

  MOV_File = './MOV/' + File + '.mp4'
  print(MOV_File)

  # 시간 좌표 토대로 영상 자르기 
  clip = mp.VideoFileClip(MOV_File)
  video_clip = clip.subclip(0, target_sec)
  video_clip.write_videofile('./CUT_VID/' + File + '.mp4')
  # 얼굴 1명 인식 기준으로 잘랐을 때 발생하였던 소리 자체가 안 담기는 현상은 해결되었음.
  # 다만 소리 잡히는 도중에 화면 전환되는 경우가 새로 발생하였음...(회의 초반에 우려하였던 문제였는데..!)
  # 이걸 대체 어떤 식으로 해결해야 할까...??
  
  clip2 = mp.VideoFileClip('./CUT_VID/' + File + '.mp4')
  clip2.audio.write_audiofile('./CUT_WAV/' + File + '.wav')

  # 대본 파일
  audio = sr.AudioFile('./CUT_WAV/' + File + '.wav')

  recognizer = sr.Recognizer()
  recognizer.energy_threshold = 300
  
  with audio as source:
    audio_news = recognizer.record(source)

  text = recognizer.recognize_google(audio_data=audio_news, language="ko-KR", show_all=False)

  f = open('./CUT_TXT/' + File + '.txt', 'w')
  text = str(text)
  f.write(text)
  f.close()

  # 이 부분 파일이 손상되어서 다시 손을 봐야할 것 같음... 끄아...
  face_crop_vid(File)


## [ 8/7~8/9 코드 작업 근황 - len(faces) 관련 아이디어 ] 
# len(faces)에서 1 사이에 낀 0을 모조리 1로 변환하려는 작업을 시도하였으나...!
# 연산량이 오바적으로 많아져서 철수했음... 눈물 뚝뚝;;
# 고작 일부분을 바꾸겠다고 비디오 파일 전체를 연산한다는 것 자체가 진짜 오바였음...

# 결론 : 그건 일단 뒤로 미룰려고 합니다... 다른 영역부터 고치고 시도해볼게요...

./WAV/readnaversid1750oid448aid0000195576.wav
./WAV/readnaversid1750oid448aid0000195576.wav : 13.16 sec
./MOV/readnaversid1750oid448aid0000195576.mp4
[MoviePy] >>>> Building video ./CUT_VID/readnaversid1750oid448aid0000195576.mp4
[MoviePy] Writing audio in readnaversid1750oid448aid0000195576TEMP_MPY_wvf_snd.mp3


100%|██████████| 291/291 [00:00<00:00, 747.79it/s]

[MoviePy] Done.
[MoviePy] Writing video ./CUT_VID/readnaversid1750oid448aid0000195576.mp4



100%|██████████| 395/395 [00:07<00:00, 54.32it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./CUT_VID/readnaversid1750oid448aid0000195576.mp4 

[MoviePy] Writing audio in ./CUT_WAV/readnaversid1750oid448aid0000195576.wav


...

[MoviePy] Done.
[MoviePy] Writing video ./CUT_VID/readnaversid1742oid437aid0000011423.mp4



100%|██████████| 241/241 [00:02<00:00, 91.84it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./CUT_VID/readnaversid1742oid437aid0000011423.mp4 

[MoviePy] Writing audio in ./CUT_WAV/readnaversid1742oid437aid0000011423.wav


100%|██████████| 223/223 [00:00<00:00, 328.98it/s]

[MoviePy] Done.
